In [ ]:
import pickle as pkl
import geopandas as gpd
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pygeos.io import from_shapely

In [ ]:
subgraph_id = 1

In [ ]:
subgraphs = pkl.load(open("../subgraphs.pkl", "rb"))
remaining_roads = pkl.load(open("../remaining_roads.pkl", "rb"))
remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads})

In [ ]:
subgraph = subgraphs[subgraph_id]
df = pd.DataFrame.from_dict(dict(subgraph.nodes(data=True)), orient='index').reset_index()
df["geometry"] = df.index
gdf = gpd.GeoDataFrame(df, geometry="index")
gdf.columns

In [ ]:
axis_buffer = 10

def get_ax(gdf):
    f, ax = plt.subplots(figsize=[15, 15])
    bounds = gdf.total_bounds
    ax.set_xlim(bounds[0] - axis_buffer, bounds[2] + axis_buffer)
    ax.set_ylim(bounds[1] - axis_buffer, bounds[3] + axis_buffer)
    return f, ax

In [ ]:


remaining_roads_gdf.plot(color="gray", ax=ax)
gdf.plot(color="green", ax=ax)
gdf.plot("interior_closure", cmap="bwr_r", ax=ax)
gdf.plot("exterior_closure", cmap="bwr", ax=ax)

In [ ]:
sources = from_shapely(gdf[gdf['interior_closure'] == True].geometry)
sinks = from_shapely(gdf[gdf['exterior_closure'] == True].geometry)
centrality = nx.algorithms.centrality.betweenness_centrality_subset(subgraph, sources, sinks)
betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(nx.to_undirected(subgraph), sources, sinks)
from pygeos.creation import linestrings
from pygeos.coordinates import get_coordinates
from pygeos.geometry import get_point
# print([linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())])
edges = [linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())]
betweeness_gdf = gpd.GeoDataFrame({"geometry": edges, "betweeness": betweeness.values()})
print(betweeness_gdf)
# centrality = nx.algorithms.cuts.cut_size(subgraph, sources, sinks)
print(centrality)
df = pd.DataFrame.from_dict(centrality, orient='index')
print(df)
df["geometry"] = df.index
centrality_gdf = gpd.GeoDataFrame(df)

In [ ]:
f, ax = get_ax(centrality_gdf)
remaining_roads_gdf.plot(color="gray", ax=ax)
centrality_gdf.plot(0, ax=ax)
gdf.plot("interior_closure", cmap="bwr_r", ax=ax)
gdf.plot("exterior_closure", cmap="bwr", ax=ax)


In [ ]:
f, ax = get_ax(centrality_gdf)
remaining_roads_gdf.plot(color="gray", ax=ax)
betweeness_gdf.plot("betweeness", ax=ax)
gdf.plot("interior_closure", cmap="bwr_r", ax=ax)
gdf.plot("exterior_closure", cmap="bwr", ax=ax)
betweeness_gdf.set_index("geometry", inplace=True)


In [ ]:
ud_subgraph = nx.Graph(nx.to_undirected(subgraph))
max_edge = from_shapely(betweeness_gdf.idxmax())
max_edge = get_point(max_edge[0], 0), get_point(max_edge[0], 1)
ud_subgraph.remove_edge(*max_edge)
betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(ud_subgraph, sources, sinks)
# print([linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())])
edges = [linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())]
betweeness_gdf = gpd.GeoDataFrame({"geometry": edges, "betweeness": betweeness.values()})
f, ax = get_ax(centrality_gdf)
remaining_roads_gdf.plot(color="gray", ax=ax)
betweeness_gdf.plot("betweeness", ax=ax)
gdf.plot("interior_closure", cmap="bwr_r", ax=ax)
gdf.plot("exterior_closure", cmap="bwr", ax=ax)
betweeness_gdf.set_index("geometry", inplace=True)

In [ ]:
ud_subgraph = nx.Graph(nx.to_undirected(ud_subgraph))
max_edge = from_shapely(betweeness_gdf.idxmax())
print(type(max_edge[0]))
max_edge = get_point(max_edge[0], 0), get_point(max_edge[0], 1)
ud_subgraph.remove_edge(*max_edge)
betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(ud_subgraph, sources, sinks)
# print([linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())])
edges = [linestrings(get_coordinates(coords)) for coords in list(betweeness.keys())]
betweeness_gdf = gpd.GeoDataFrame({"geometry": edges, "betweeness": betweeness.values()})
print(betweeness_gdf)
f, ax = get_ax(centrality_gdf)
remaining_roads_gdf.plot(color="gray", ax=ax)
betweeness_gdf.plot("betweeness", ax=ax)
gdf.plot("interior_closure", cmap="bwr_r", ax=ax)
gdf.plot("exterior_closure", cmap="bwr", ax=ax)
betweeness_gdf.set_index("geometry", inplace=True)

In [ ]:
subgraph_id_gen = (i for i in subgraphs.keys())

In [ ]:
# subgraph_id = next(subgraph_id_gen)
subgraph = subgraphs[3]
# print(subgraph_id)
ud_subgraph = nx.Graph(nx.to_undirected(subgraph))
sources = [node[0] for node in ud_subgraph.nodes("interior_closure") if node[1]]
sinks = [node[0] for node in ud_subgraph.nodes("exterior_closure") if node[1]]
betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(ud_subgraph, sources, sinks)
removed_edges = []

while any(set(sinks) < set(nx.bfs_tree(ud_subgraph, source).nodes()) for source in sources):
    max_edge = max(betweeness.items(), key=operator.itemgetter(1))
    ud_subgraph.remove_edge(*max_edge[0])
    removed_edges.append(max_edge)
    try:
        betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(ud_subgraph, sources, sinks)
    except:
        break

final_edges = [linestrings(get_coordinates(coords)) for coords in ud_subgraph.edges]
starting_edges = [linestrings(get_coordinates(coords)) for coords in subgraph.edges]
final_edges_gdf = gpd.GeoDataFrame({"geometry": final_edges})
starting_edges_gdf = gpd.GeoDataFrame({"geometry": starting_edges})
sources_gdf = gpd.GeoDataFrame({"geometry": sources})
sinks_gdf = gpd.GeoDataFrame({"geometry": sinks})

f, ax = get_ax(starting_edges_gdf)
remaining_roads_gdf.plot(color="gray", ax=ax)
starting_edges_gdf.plot(ax=ax, color="r")
final_edges_gdf.plot(ax=ax)
sources_gdf.plot(color="r", ax=ax)
sinks_gdf.plot(color="g", ax=ax)

In [ ]:
import operator

max_edge = max(betweeness.items(), key=operator.itemgetter(1))

In [ ]:
def calculate_betweeness(graph, graph_holder, removed_edges):
    sources = [node[0] for node in graph.nodes("interior_closure") if node[1]]
    sinks = [node[0] for node in graph.nodes("exterior_closure") if node[1]]
    if sources and sinks:
        betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(graph, sources, sinks)
        max_edge = max(betweeness.items(), key=operator.itemgetter(1))
        graph.remove_edge(*max_edge[0])
        removed_edges.append(max_edge)
        if any(set(sinks) < set(nx.bfs_tree(graph, source).nodes()) for source in sources):
            for component in nx.connected_components(graph):
                graph_holder.append(graph)
                return calculate_betweeness(graph.subgraph(list(component)).copy(), graph_holder, removed_edges)
        else:
            graph_holder.append(graph)
            return graph, graph_holder, removed_edges
    else:
        return graph, graph_holder, removed_edges
    
    
# def find_max_edge()
#     betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(graph, sources, sinks)
#     max_edge = max(betweeness.items(), key=operator.itemgetter(1))

In [ ]:
from collections import deque


def get_disconnected_edges(graph):
    graph_deque = deque([graph.copy()])
    disconnected = []
    while len(graph_deque):
        g = graph_deque.pop()
        sources = [node[0] for node in g.nodes("interior_closure") if node[1]]
        sinks = [node[0] for node in g.nodes("exterior_closure") if node[1]]
        if sources and sinks:
            betweeness = nx.algorithms.centrality.edge_current_flow_betweenness_centrality_subset(g, sources, sinks)
            max_edge = max(betweeness.items(), key=operator.itemgetter(1))
            g.remove_edge(*max_edge[0])
            reachable = []
            for source in sources:
                reachable.extend(nx.dfs_tree(g, source).nodes())
            if set(sinks).intersection(set(reachable)):
                for component in nx.connected_components(g):
                    graph_deque.append(g.subgraph(list(component)).copy())
            else:
                disconnected.extend(g.edges)
        else:
            disconnected.extend(g.edges)
#     return disconnected

In [ ]:
subgraph_id_gen = (i for i in subgraphs.keys())

In [ ]:
# from AutoCordon.prune_edges import get_disconnected_edges

subgraphs = pkl.load(open("../subgraphs.pkl", "rb"))
remaining_roads = pkl.load(open("../remaining_roads.pkl", "rb"))
remaining_roads_gdf = gpd.GeoDataFrame({"geometry": remaining_roads})
for subgraph_id in subgraphs:
    graph = nx.Graph(nx.to_undirected(subgraphs[subgraph_id]))
    sources = [node[0] for node in graph.nodes("interior_closure") if node[1]]
    sinks = [node[0] for node in graph.nodes("exterior_closure") if node[1]]
    sources_gdf = gpd.GeoDataFrame({"geometry": sources})
    sinks_gdf = gpd.GeoDataFrame({"geometry": sinks})
    starting_edges = [linestrings(get_coordinates(coords)) for coords in graph.edges]
    starting_edges_gdf = gpd.GeoDataFrame({"geometry": starting_edges})
    
    f, ax = get_ax(starting_edges_gdf)
    remaining_roads_gdf.plot(color="gray", ax=ax)
    starting_edges_gdf.plot(ax=ax, color="r")
    sources_gdf.plot(color="r", ax=ax)
    sinks_gdf.plot(color="g", ax=ax)
    ax.set_title(subgraph_id)
    
#     try:
    disconnected = get_disconnected_edges(graph)
    final_edges = [linestrings(get_coordinates(coords)) for coords in disconnected]
    final_edges_gdf = gpd.GeoDataFrame({"geometry": final_edges})
    final_edges_gdf.plot(ax=ax)
#     except:
#         print(f"skipping subgraph {subgraph_id}")
    plt.show()
    